<a href="https://colab.research.google.com/github/Santosdevbjj/generosLiterarios/blob/main/GenerosLivros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:

# Célula 2: Definição dos Modelos e da Lógica de Banco de Dados (DAO)

import mysql.connector
from mysql.connector import Error
import sys

# --- MODELOS DE DADOS ---

class Livro:
    """Classe que representa a entidade Livro no banco de dados."""
    def __init__(self, titulo, autor, ano_publicacao, genero_literario, best_seller, pais, idioma, id=None):
        self.id = id
        self.titulo = titulo
        self.autor = autor
        self.ano_publicacao = ano_publicacao
        self.genero_literario = genero_literario
        self.best_seller = 1 if best_seller in (True, 1, 's', 'S') else 0
        self.pais = pais
        self.idioma = idioma

    def __str__(self):
        best_seller_str = "Sim" if self.best_seller == 1 else "Não"
        return (f"ID: {self.id} | Título: {self.titulo} | Autor: {self.autor} | Ano: {self.ano_publicacao} | "
                f"Gênero: {self.genero_literario} | Best Seller: {best_seller_str}")


class Cliente:
    """Classe que representa a entidade Cliente."""
    def __init__(self, nome, preferencias_genero, estilo_leitura, id=None):
        self.id = id
        self.nome = nome
        self.preferencias_genero = preferencias_genero
        self.estilo_leitura = estilo_leitura

    def __str__(self):
        return (f"ID: {self.id} | Cliente: {self.nome} | Pref. Gênero: {self.preferencias_genero} | "
                f"Estilo Leitura: {self.estilo_leitura}")


class Avaliacao:
    """Classe que representa a avaliação de um livro por um cliente."""
    def __init__(self, id_cliente, id_livro, nota, comentario, id=None):
        self.id = id
        self.id_cliente = id_cliente
        self.id_livro = id_livro
        self.nota = nota
        self.comentario = comentario

    def __str__(self):
        return (f"ID: {self.id} | Cliente ID: {self.id_cliente} | Livro ID: {self.id_livro} | "
                f"Nota: {self.nota}/5 | Comentário: {self.comentario}")


# --- CLASSE GERENCIADORA DE DADOS (DAO) ---

class GerenciadorBookStylist:
    """Gerencia a conexão com o MySQL e todas as operações CRUD e de Recomendação."""
    def __init__(self, host, user, password, database):
        self.host = host
        self.user = user
        self.password = password
        self.database = database
        self.conexao = None
        self.cursor = None

    def conectar(self):
        """Estabelece a conexão com o banco de dados, criando o DB se não existir."""
        try:
            self.conexao = mysql.connector.connect(
                host=self.host,
                user=self.user,
                password=self.password,
                database=self.database
            )
            if self.conexao.is_connected():
                self.cursor = self.conexao.cursor()
                print("Conexão com o banco de dados estabelecida!")
        except Error as e:
            if '1049' in str(e): # Código de erro para 'Unknown database'
                try:
                    conn_sem_db = mysql.connector.connect(host=self.host, user=self.user, password=self.password)
                    cursor_sem_db = conn_sem_db.cursor()
                    cursor_sem_db.execute(f"CREATE DATABASE {self.database}")
                    conn_sem_db.close()

                    self.conexao = mysql.connector.connect(host=self.host, user=self.user, password=self.password, database=self.database)
                    self.cursor = self.conexao.cursor()
                    print(f"Banco de dados '{self.database}' criado e conectado!")
                except Error as e_create:
                    print(f"Erro ao criar ou conectar ao MySQL: {e_create}")
            else:
                print(f"Erro ao conectar ao MySQL: {e}")
            self.conexao = None
            self.cursor = None

    def desconectar(self):
        """Fecha a conexão com o banco de dados."""
        if self.conexao and self.conexao.is_connected():
            self.cursor.close()
            self.conexao.close()
            print("Conexão com o banco de dados fechada.")

    def criar_tabelas(self):
        """Cria as tabelas (livros, clientes, avaliacoes) se não existirem."""
        if not self.conexao: return

        tabela_livros = """
        CREATE TABLE IF NOT EXISTS livros (
            id INT AUTO_INCREMENT PRIMARY KEY,
            titulo VARCHAR(255) NOT NULL,
            autor VARCHAR(255),
            ano_publicacao INT,
            genero_literario VARCHAR(255),
            best_seller BOOLEAN,
            pais VARCHAR(100),
            idioma VARCHAR(100)
        )
        """
        tabela_clientes = """
        CREATE TABLE IF NOT EXISTS clientes (
            id INT AUTO_INCREMENT PRIMARY KEY,
            nome VARCHAR(255) NOT NULL,
            preferencias_genero VARCHAR(500),
            estilo_leitura VARCHAR(500)
        )
        """
        tabela_avaliacoes = """
        CREATE TABLE IF NOT EXISTS avaliacoes (
            id INT AUTO_INCREMENT PRIMARY KEY,
            id_cliente INT,
            id_livro INT,
            nota INT,
            comentario TEXT,
            FOREIGN KEY (id_cliente) REFERENCES clientes(id) ON DELETE CASCADE,
            FOREIGN KEY (id_livro) REFERENCES livros(id) ON DELETE CASCADE
        )
        """
        try:
            self.cursor.execute(tabela_livros)
            self.cursor.execute(tabela_clientes)
            self.cursor.execute(tabela_avaliacoes)
            self.conexao.commit()
            print("Tabelas criadas/verificadas com sucesso.")
        except Error as e:
            print(f"Erro ao criar tabelas: {e}")

    # --- Métodos Auxiliares ---

    def _executar_query(self, query, params=None, commit=False, fetch_one=False, fetch_all=False):
        """Método auxiliar para executar queries e tratar erros."""
        if not self.conexao:
            print("Erro: Conexão com o banco de dados não está ativa.")
            return None

        try:
            self.cursor.execute(query, params or ())
            if commit:
                self.conexao.commit()
                return self.cursor.rowcount > 0 or self.cursor.lastrowid
            if fetch_one:
                return self.cursor.fetchone()
            if fetch_all:
                return self.cursor.fetchall()
            return True
        except Error as e:
            print(f"Erro na operação de banco de dados: {e}")
            return None

    # --- CRUD: Livros ---

    def adicionar_livro(self, livro: Livro):
        query = ("INSERT INTO livros (titulo, autor, ano_publicacao, genero_literario, best_seller, pais, idioma) "
                 "VALUES (%s, %s, %s, %s, %s, %s, %s)")
        valores = (livro.titulo, livro.autor, livro.ano_publicacao, livro.genero_literario,
                   livro.best_seller, livro.pais, livro.idioma)
        if self._executar_query(query, valores, commit=True):
            print(f"Livro '{livro.titulo}' adicionado.")

    def buscar_livros(self, filtro=None, valor=None):
        if filtro and valor:
            query = f"SELECT * FROM livros WHERE {filtro} LIKE %s;"
            resultados = self._executar_query(query, ('%' + str(valor) + '%',), fetch_all=True)
        else:
            query = "SELECT * FROM livros;"
            resultados = self._executar_query(query, fetch_all=True)

        if resultados:
            return [Livro(r[1], r[2], r[3], r[4], r[5], r[6], r[7], id=r[0]) for r in resultados]
        return []

    def deletar_livro(self, id_livro):
        query = "DELETE FROM livros WHERE id = %s"
        if self._executar_query(query, (id_livro,), commit=True):
            print(f"Livro ID {id_livro} deletado com sucesso.")
            return True
        return False

    # --- CRUD: Clientes ---

    def adicionar_cliente(self, cliente: Cliente):
        query = "INSERT INTO clientes (nome, preferencias_genero, estilo_leitura) VALUES (%s, %s, %s)"
        valores = (cliente.nome, cliente.preferencias_genero, cliente.estilo_leitura)
        if self._executar_query(query, valores, commit=True):
            print(f"Cliente '{cliente.nome}' adicionado.")

    def buscar_clientes(self, id_cliente=None):
        if id_cliente:
            query = "SELECT * FROM clientes WHERE id = %s"
            resultados = self._executar_query(query, (id_cliente,), fetch_all=True)
        else:
            query = "SELECT * FROM clientes"
            resultados = self._executar_query(query, fetch_all=True)

        if resultados:
            return [Cliente(r[1], r[2], r[3], id=r[0]) for r in resultados]
        return []

    def deletar_cliente(self, id_cliente):
        query = "DELETE FROM clientes WHERE id = %s"
        if self._executar_query(query, (id_cliente,), commit=True):
            print(f"Cliente ID {id_cliente} deletado com sucesso.")
            return True
        return False

    # --- CRUD: Avaliações ---

    def adicionar_avaliacao(self, avaliacao: Avaliacao):
        query = "INSERT INTO avaliacoes (id_cliente, id_livro, nota, comentario) VALUES (%s, %s, %s, %s)"
        valores = (avaliacao.id_cliente, avaliacao.id_livro, avaliacao.nota, avaliacao.comentario)
        if self._executar_query(query, valores, commit=True):
            print("Avaliação adicionada com sucesso.")

    def buscar_avaliacoes(self, id_cliente=None):
        if id_cliente:
            query = "SELECT * FROM avaliacoes WHERE id_cliente = %s"
            resultados = self._executar_query(query, (id_cliente,), fetch_all=True)
        else:
            query = "SELECT * FROM avaliacoes"
            resultados = self._executar_query(query, fetch_all=True)

        if resultados:
            return [Avaliacao(r[1], r[2], r[3], r[4], id=r[0]) for r in resultados]
        return []

    # --- LÓGICA DE RECOMENDAÇÃO ---

    def recomendar_livros(self, id_cliente):
        """Gera uma recomendação híbrida: Gênero Preferido + Popularidade, excluindo os lidos."""
        cliente_obj = self.buscar_clientes(id_cliente)
        if not cliente_obj:
            print(f"Cliente com ID {id_cliente} não encontrado.")
            return []

        cliente = cliente_obj[0]
        generos_pref = [g.strip() for g in cliente.preferencias_genero.split(',') if g.strip()]

        # 1. Encontrar livros que o cliente JÁ avaliou (para excluí-los)
        query_avaliados = "SELECT id_livro FROM avaliacoes WHERE id_cliente = %s"
        livros_avaliados = self._executar_query(query_avaliados, (id_cliente,), fetch_all=True)
        ids_avaliados = [item[0] for item in livros_avaliados] if livros_avaliados else [0]
        ids_avaliados_str = ', '.join(map(str, ids_avaliados)) # Usado para o NOT IN na query

        # 2. Query principal de recomendação (Filtro por Gênero e Popularidade)
        params = []
        if generos_pref:
            placeholders = ', '.join(['%s'] * len(generos_pref))
            params = generos_pref

            query_recomendacao = f"""
            SELECT
                l.id, l.titulo, l.autor, AVG(a.nota) as media_nota
            FROM
                livros l
            LEFT JOIN
                avaliacoes a ON l.id = a.id_livro
            WHERE
                l.genero_literario IN ({placeholders}) AND l.id NOT IN ({ids_avaliados_str})
            GROUP BY
                l.id, l.titulo, l.autor
            HAVING
                media_nota >= 3.5
            ORDER BY
                media_nota DESC
            LIMIT 5;
            """
        else:
            # Sugere Best Sellers populares se não houver gênero preferido
            query_recomendacao = f"""
            SELECT
                l.id, l.titulo, l.autor, AVG(a.nota) as media_nota
            FROM
                livros l
            LEFT JOIN
                avaliacoes a ON l.id = a.id_livro
            WHERE
                l.best_seller = 1 AND l.id NOT IN ({ids_avaliados_str})
            GROUP BY
                l.id, l.titulo, l.autor
            HAVING
                media_nota >= 3.5
            ORDER BY
                media_nota DESC
            LIMIT 5;
            """
            # params permanece vazio, pois não há placeholders de %s na query.

        recomendacoes = self._executar_query(query_recomendacao, params, fetch_all=True)

        if not recomendacoes:
            # Lógica alternativa: Sugerir livros mais bem avaliados em geral (Fallback)
            print("Não foi possível encontrar recomendações perfeitas. Sugerindo os mais populares.")
            query_fallback = f"""
            SELECT l.id, l.titulo, l.autor, AVG(a.nota) as media_nota FROM livros l
            LEFT JOIN avaliacoes a ON l.id = a.id_livro
            WHERE l.id NOT IN ({ids_avaliados_str})
            GROUP BY l.id, l.titulo, l.autor HAVING media_nota IS NOT NULL
            ORDER BY media_nota DESC LIMIT 5;
            """
            recomendacoes = self._executar_query(query_fallback, fetch_all=True)

        if not recomendacoes:
             return ["Nenhuma recomendação disponível. Adicione mais livros e avaliações."]

        return [f"ID: {r[0]} | Título: {r[1]} | Autor: {r[2]} | Nota Média: {r[3]:.2f}" for r in recomendacoes]

In [1]:

# Célula 1: Instalação e Configuração do MySQL no Colab

# 1. Instala o MySQL server e o cliente
# `-y` para confirmar automaticamente a instalação
!apt-get install -y -qq mysql-server &> /dev/null
print("MySQL server instalado com sucesso.")

# 2. Inicia o serviço do MySQL
!service mysql start
print("MySQL server iniciado.")

# 3. Instala a biblioteca Python para conectar ao MySQL
!pip install -q mysql-connector-python
print("Biblioteca 'mysql-connector-python' instalada.")

# 4. Define um usuário e senha para o banco de dados
# O usuário é necessário para acesso do Python.
!mysql -e "CREATE USER 'usuario_crud'@'localhost' IDENTIFIED BY 'senha123';"
!mysql -e "GRANT ALL PRIVILEGES ON *.* TO 'usuario_crud'@'localhost' WITH GRANT OPTION;"
!mysql -e "FLUSH PRIVILEGES;"
print("Usuário 'usuario_crud' criado e privilégios concedidos.")

MySQL server instalado com sucesso.
 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.
MySQL server iniciado.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 62.5 MB/s eta 0:00:00
Biblioteca 'mysql-connector-python' instalada.
Usuário 'usuario_crud' criado e privilégios concedidos.


In [6]:

# Célula 3: Interface de Usuário e Execução Principal

# --- Funções Auxiliares de Tratamento de Erro ---

def _input_int(prompt):
    """Função auxiliar para garantir que a entrada seja um número inteiro."""
    while True:
        try:
            return int(input(prompt))
        except ValueError:
            print("⚠️ Entrada inválida. Por favor, digite um número inteiro.")

def _input_sim_nao(prompt):
    """Função auxiliar para garantir que a entrada seja 's' ou 'n'."""
    while True:
        valor = input(prompt).lower()
        if valor in ('s', 'n'):
            return valor == 's'
        print("⚠️ Entrada inválida. Digite 's' para Sim ou 'n' para Não.")


# --- Menus de Gerenciamento ---

def menu_livros(gerenciador: GerenciadorBookStylist):
    """Menu para operações de CRUD de Livros."""
    while True:
        print("\n--- Gerenciamento de Livros ---")
        print("1. Adicionar Livro (CREATE)")
        print("2. Listar Todos (READ)")
        print("3. Buscar por Título/Autor/Gênero")
        print("4. Deletar Livro (DELETE)")
        print("5. Voltar ao Menu Principal")
        escolha = input("Escolha uma opção: ")

        if escolha == '1':
            try:
                titulo = input("Título: ")
                autor = input("Autor: ")
                ano = _input_int("Ano de Publicação: ")
                genero = input("Gênero Literário: ")
                best_seller = _input_sim_nao("É um Best Seller? (s/n): ")
                pais = input("País: ")
                idioma = input("Idioma: ")
                novo_livro = Livro(titulo, autor, ano, genero, best_seller, pais, idioma)
                gerenciador.adicionar_livro(novo_livro)
            except Exception as e:
                print(f"❌ Erro ao adicionar livro: {e}")

        elif escolha == '2':
            livros = gerenciador.buscar_livros()
            if livros:
                 print("\n--- Todos os Livros ---")
                 for livro in livros:
                     print(livro)
            else:
                print("Nenhum livro cadastrado.")

        elif escolha == '3':
            filtro = input("Buscar por (titulo/autor/genero_literario): ")
            valor = input(f"Valor a buscar em {filtro}: ")
            livros = gerenciador.buscar_livros(filtro, valor)
            if livros:
                 print("\n--- Livros Encontrados ---")
                 for livro in livros:
                     print(livro)
            else:
                print("Nenhum livro encontrado.")

        elif escolha == '4':
            id_deletar = _input_int("Digite o ID do livro para deletar: ")
            gerenciador.deletar_livro(id_deletar)

        elif escolha == '5':
            break

def menu_clientes(gerenciador: GerenciadorBookStylist):
    """Menu para operações de CRUD de Clientes e Avaliações."""
    while True:
        print("\n--- Gerenciamento de Clientes e Avaliações ---")
        print("1. Adicionar Novo Cliente (CREATE)")
        print("2. Listar Clientes (READ)")
        print("3. Deletar Cliente (DELETE)")
        print("4. Adicionar Avaliação")
        print("5. Listar Avaliações de um Cliente")
        print("6. Voltar ao Menu Principal")
        escolha = input("Escolha uma opção: ")

        if escolha == '1':
            nome = input("Nome do Cliente: ")
            generos = input("Gêneros Favoritos (separados por vírgula, Ex: Ficção, Romance): ")
            estilo = input("Estilo de Leitura (Ex: rápido e ação, lento e reflexivo): ")
            novo_cliente = Cliente(nome, generos, estilo)
            gerenciador.adicionar_cliente(novo_cliente)

        elif escolha == '2':
            clientes = gerenciador.buscar_clientes()
            if clientes:
                print("\n--- Clientes Cadastrados ---")
                for cliente in clientes:
                    print(cliente)
            else:
                print("Nenhum cliente cadastrado.")

        elif escolha == '3':
            id_deletar = _input_int("Digite o ID do cliente para deletar: ")
            gerenciador.deletar_cliente(id_deletar)

        elif escolha == '4':
            try:
                id_cliente = _input_int("ID do Cliente que está avaliando: ")
                id_livro = _input_int("ID do Livro avaliado: ")
                nota = _input_int("Nota (1 a 5): ")
                comentario = input("Comentário: ")

                if 1 <= nota <= 5:
                    nova_avaliacao = Avaliacao(id_cliente, id_livro, nota, comentario)
                    gerenciador.adicionar_avaliacao(nova_avaliacao)
                else:
                    print("⚠️ Nota inválida. Deve ser entre 1 e 5.")
            except Exception as e:
                print(f"❌ Erro ao adicionar avaliação. Verifique se o Cliente e o Livro existem: {e}")

        elif escolha == '5':
            id_cliente = _input_int("ID do Cliente para ver avaliações: ")
            avaliacoes = gerenciador.buscar_avaliacoes(id_cliente)
            if avaliacoes:
                print(f"\n--- Avaliações do Cliente ID {id_cliente} ---")
                for av in avaliacoes:
                    print(av)
            else:
                print(f"Cliente ID {id_cliente} não possui avaliações.")

        elif escolha == '6':
            break

# --- Função Principal ---

def main():
    """Função principal que inicia e gerencia o loop do programa."""
    print("Iniciando o Sistema Book Stylist...")

    # Configuração de conexão consistente com o ambiente Colab
    gerenciador = GerenciadorBookStylist(
        host="localhost",
        user="usuario_crud",
        password="senha123",
        database="book_stylist_db"
    )

    gerenciador.conectar()
    if gerenciador.conexao is None:
        print("❌ Falha na conexão inicial com o banco de dados. Encerrando.")
        return

    gerenciador.criar_tabelas()

    while True:
        print("\n--- Menu Principal ---")
        print("1. Gerenciar Livros (CRUD)")
        print("2. Gerenciar Clientes e Avaliações")
        print("3. Gerar Recomendação (Inteligência Book Stylist)")
        print("4. Sair")
        escolha = input("Escolha uma opção: ")

        if escolha == '1':
            menu_livros(gerenciador)

        elif escolha == '2':
            menu_clientes(gerenciador)

        elif escolha == '3':
            print("\n--- Gerar Recomendação ---")
            id_cliente = _input_int("Digite o ID do Cliente para recomendação: ")
            recomendacoes = gerenciador.recomendar_livros(id_cliente)

            if isinstance(recomendacoes, list) and recomendacoes[0].startswith("Nenhuma recomendação"):
                 print(recomendacoes[0])
            elif recomendacoes:
                print("\n✅ As 5 Melhores Recomendações Book Stylist:")
                for i, rec in enumerate(recomendacoes):
                    print(f"{i+1}. {rec}")

        elif escolha == '4':
            print("Encerrando o Sistema. Até logo!")
            gerenciador.desconectar()
            break

        else:
            print("⚠️ Opção inválida. Tente novamente.")

# Executa a função principal para iniciar o sistema no Colab
if __name__ == "__main__":
    main()

Iniciando o Sistema Book Stylist...
Conexão com o banco de dados estabelecida!
Tabelas criadas/verificadas com sucesso.

--- Menu Principal ---
1. Gerenciar Livros (CRUD)
2. Gerenciar Clientes e Avaliações
3. Gerar Recomendação (Inteligência Book Stylist)
4. Sair
Escolha uma opção: 1

--- Gerenciamento de Livros ---
1. Adicionar Livro (CREATE)
2. Listar Todos (READ)
3. Buscar por Título/Autor/Gênero
4. Deletar Livro (DELETE)
5. Voltar ao Menu Principal
Escolha uma opção: 2
Nenhum livro cadastrado.

--- Gerenciamento de Livros ---
1. Adicionar Livro (CREATE)
2. Listar Todos (READ)
3. Buscar por Título/Autor/Gênero
4. Deletar Livro (DELETE)
5. Voltar ao Menu Principal
Escolha uma opção: 1
Título: 5
Autor: 4
Ano de Publicação: 5


KeyboardInterrupt: Interrupted by user